In [17]:
import os
import pandas as pd
import numpy as np
import zipfile
import tensorflow
from sklearn.model_selection import train_test_split

In [3]:
# load zip files into colab session, in order to extract data in session and then load files speedy for training and testing, because upload and mounting of google drive was not speedy at all
with zipfile.ZipFile('/content/data_fabi-20250112T105723Z-001.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data_dir_1')

with zipfile.ZipFile('/content/data-gesamt-20250112T110540Z-001.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data_dir_2')

data_dir_1 = '/content/data_dir_1'
data_dir_2 = '/content/data_dir_2'

In [6]:
# merge data
folder_mapping = {
    'data-gesamt/Telefon': ['data_fabi/telefon_nl', 'data_fabi/telefon'],
    'data-gesamt/Peace': ['data_fabi/peace_nl', 'data_fabi/peace'],
    'data-gesamt/Open-Hand': ['data_fabi/openhand_nl', 'data_fabi/openhand'],
    'data-gesamt/Faust': ['data_fabi/faust_nl', 'data_fabi/faust'],
    'data-gesamt/Thumb-Down': ['data_fabi/daumenrunter_nl', 'data_fabi/daumenrunter'],
    'data-gesamt/Thumb-Up': ['data_fabi/daumenhoch', 'data_fabi/daumenhoch_nl']
}

def read_files_to_df(folder_path):
    data = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            data.append({
                'file_name': file,
                'file_path': file_path,
                'folder': os.path.basename(root)
            })
    return pd.DataFrame(data)

merged_df = pd.DataFrame()

# read data_dir_2 and map to categories
for folder in folder_mapping.keys():
    folder_path = os.path.join(data_dir_2, folder)
    if os.path.exists(folder_path):
        df = read_files_to_df(folder_path)
        df['category'] = folder
        merged_df = pd.concat([merged_df, df])

# read data_dir_1 and map to categories
for dest_folder, src_folders in folder_mapping.items():
    for src_folder in src_folders:
        folder_path = os.path.join(data_dir_1, src_folder)
        if os.path.exists(folder_path):
            df = read_files_to_df(folder_path)
            df['category'] = dest_folder
            merged_df = pd.concat([merged_df, df])

In [7]:
merged_df

,file_name,file_path,folder,category
0,b_r_40_n_nl.npy,/content/data_dir_2/data-gesamt/Telefon/b_r_40...,Telefon,data-gesamt/Telefon
1,b_l_91_a_nl.npy,/content/data_dir_2/data-gesamt/Telefon/b_l_91...,Telefon,data-gesamt/Telefon
2,b_l_15_a_wl.npy,/content/data_dir_2/data-gesamt/Telefon/b_l_15...,Telefon,data-gesamt/Telefon
3,b_r_86_n_nl.npy,/content/data_dir_2/data-gesamt/Telefon/b_r_86...,Telefon,data-gesamt/Telefon
4,b_r_11_n_wl.npy,/content/data_dir_2/data-gesamt/Telefon/b_r_11...,Telefon,data-gesamt/Telefon
...,...,...,...,...
395,f_l_87_a_nl.npy,/content/data_dir_1/data_fabi/daumenhoch_nl/f_...,daumenhoch_nl,data-gesamt/Thumb-Up
396,f_l_77_a_nl.npy,/content/data_dir_1/data_fabi/daumenhoch_nl/f_...,daumenhoch_nl,data-gesamt/Thumb-Up
397,f_l_74_a_nl.npy,/content/data_dir_1/data_fabi/daumenhoch_nl/f_...,daumenhoch_nl,data-gesamt/Thumb-Up
398,f_r_82_c_nl.npy,/content/data_dir_1/data_fabi/daumenhoch_nl/f_...,daumenhoch_nl,data-gesamt/Thumb-Up


In [9]:
# clean category-concatination-string of category column in merged dataframe
merged_df['category'] = merged_df['category'].str.split('/').str.get(-1)

In [10]:
# check if each category is fully integrated - need 1600 samples per category
print(merged_df['category'].value_counts())

category
Telefon       1600
Peace         1600
Open-Hand     1600
Faust         1600
Thumb-Down    1600
Thumb-Up      1600
Name: count, dtype: int64


In [12]:
# now the data is in some sort of order because of the merging, and for training and testing, we need to shuffle it per category
merged_df[merged_df['category'] == 'Faust']

,file_name,file_path,folder,category
0,b_r_40_n_nl.npy,/content/data_dir_2/data-gesamt/Faust/b_r_40_n...,Faust,Faust
1,b_l_91_a_nl.npy,/content/data_dir_2/data-gesamt/Faust/b_l_91_a...,Faust,Faust
2,b_l_15_a_wl.npy,/content/data_dir_2/data-gesamt/Faust/b_l_15_a...,Faust,Faust
3,b_r_86_n_nl.npy,/content/data_dir_2/data-gesamt/Faust/b_r_86_n...,Faust,Faust
4,b_r_11_n_wl.npy,/content/data_dir_2/data-gesamt/Faust/b_r_11_n...,Faust,Faust
...,...,...,...,...
395,f_l_42_c_wl.npy,/content/data_dir_1/data_fabi/faust/f_l_42_c_w...,faust,Faust
396,f_l_96_c_wl.npy,/content/data_dir_1/data_fabi/faust/f_l_96_c_w...,faust,Faust
397,f_r_66_c_wl.npy,/content/data_dir_1/data_fabi/faust/f_r_66_c_w...,faust,Faust
398,f_r_9_c_wl.npy,/content/data_dir_1/data_fabi/faust/f_r_9_c_wl...,faust,Faust


In [13]:
# randomise entries per category
merged_df = merged_df.groupby('category').apply(lambda x: x.sample(frac=1)).reset_index(drop=True)

# check if order is now mixed
merged_df[merged_df['category'] == 'Faust']

<ipython-input-13-cb03220f0d2f>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby('category').apply(lambda x: x.sample(frac=1)).reset_index(drop=True)


,file_name,file_path,folder,category
0,f_l_45_c_wl.npy,/content/data_dir_1/data_fabi/faust/f_l_45_c_w...,faust,Faust
1,f_l_73_c_wl.npy,/content/data_dir_1/data_fabi/faust/f_l_73_c_w...,faust,Faust
2,b_l_96_n_wl.npy,/content/data_dir_2/data-gesamt/Faust/b_l_96_n...,Faust,Faust
3,f_l_29_a_wl.npy,/content/data_dir_1/data_fabi/faust/f_l_29_a_w...,faust,Faust
4,f_r_29_a_wl.npy,/content/data_dir_1/data_fabi/faust/f_r_29_a_w...,faust,Faust
...,...,...,...,...
1595,f_l_67_a_nl.npy,/content/data_dir_1/data_fabi/faust_nl/f_l_67_...,faust_nl,Faust
1596,b_r_56_n_wl.npy,/content/data_dir_2/data-gesamt/Faust/b_r_56_n...,Faust,Faust
1597,b_l_62_a_nl.npy,/content/data_dir_2/data-gesamt/Faust/b_l_62_a...,Faust,Faust
1598,f_r_36_a_nl.npy,/content/data_dir_1/data_fabi/faust_nl/f_r_36_...,faust_nl,Faust


In [18]:
# prepare data for training and testing
X = []
y = []

# map categories to numerical values
gestures = ['Faust', 'Peace', 'Thumb-Up', 'Thumb-Down', 'Open-Hand', 'Telefon']
category_to_label = {category: i for i, category in enumerate(gestures)}

for _, row in merged_df.iterrows():
    try:
        landmarks = np.load(row['file_path'])
        X.append(landmarks)
        y.append(category_to_label[row['category']])
    except Exception as e:
        print(f"Fehler beim Laden der Datei {row['file_path']}: {e}")

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [19]:
# evalute shapes
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# evaluate distribution
for i, gesture in enumerate(gestures):
    print(f"{gesture}: {np.sum(y_train == i)} in Trainingsdaten, {np.sum(y_test == i)} in Testdaten")

X_train shape: (6720, 63)
X_test shape: (2880, 63)
y_train shape: (6720,)
y_test shape: (2880,)
Faust: 1120 in Trainingsdaten, 480 in Testdaten
Peace: 1120 in Trainingsdaten, 480 in Testdaten
Thumb-Up: 1120 in Trainingsdaten, 480 in Testdaten
Thumb-Down: 1120 in Trainingsdaten, 480 in Testdaten
Open-Hand: 1120 in Trainingsdaten, 480 in Testdaten
Telefon: 1120 in Trainingsdaten, 480 in Testdaten


In [21]:
# for building the layers of our model, we need to know the shape (63)
print("Shape of X_train:", X_train.shape)


Shape of X_train: (6720, 63)


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(63, 1)), 
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Conv1D(64, 3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.3)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.2131 - loss: 1.7559 - val_accuracy: 0.6791 - val_loss: 0.9763
Epoch 2/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5970 - loss: 0.9486 - val_accuracy: 0.7966 - val_loss: 0.4792
Epoch 3/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7781 - loss: 0.5670 - val_accuracy: 0.8547 - val_loss: 0.3803
Epoch 4/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8168 - loss: 0.4569 - val_accuracy: 0.8780 - val_loss: 0.3241
Epoch 5/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8305 - loss: 0.4275 - val_accuracy: 0.8829 - val_loss: 0.2994
Epoch 6/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8564 - loss: 0.3817 - val_accuracy: 0.8725 - val_loss: 0.3153
Epoch 7/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8527 - loss: 0.3731 - val_accuracy: 0.8924 - val_loss: 0.2769
Epoch 8/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8650 - loss: 0.3496 - val_accuracy: 0

In [24]:
# save the model
model.save('/content/' + 'B_Model_50epochs_acc_097.hdf5')

In [25]:
Ftest_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9807 - loss: 0.0724
Test accuracy: 0.9854166507720947


The following code is then for live-testing of the trained model (locally to be able to start opencv camera and get live feedback which gesture you show)

In [ ]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 56.4 MB/s eta 0:00:00


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow
from tensorflow.keras.models import load_model

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

model = load_model(f'path-to-model')
gestures = ['Faust', 'Peace', 'Thumb-Up', 'Thumb-Down', 'Open-Hand', 'Telefon']

cap = cv2.VideoCapture(0)
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])

            landmarks = np.array(landmarks).reshape(1, 63, 1)
            prediction = model.predict(landmarks)
            gesture_index = np.argmax(prediction)
            gesture = gestures[gesture_index]

            cv2.putText(frame, gesture, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Hand Gesture Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
